In [83]:
import numpy as np
A = np.array([[1,1,0,0],[1,0,1,0],[1,1,0,0],[1,1,1,1],[0,0,0,0],[1,0,1,1],[0,0,1,1]],dtype=np.bool)
#m is the number of training examples (rows in A)
m = np.shape(A)[0]
#n is the number of variables (columns in A)
n = np.shape(A)[1]
#initialize the mutual information MI, a nxn square matrix with zeros
MI = np.zeros((n, n))
#get the indexes of the triangular matrix with 1 offset
index_of_tri = np.triu_indices(n,1)
#put what you want in an array and introduced those values in the index of the matrix
MI[index_of_tri] = [1,2,3,4,5,6]
#add that triangle matrix to it own transpose, the you have the symetric matrix
MI = MI+MI.T
MI

array([[0., 1., 2., 3.],
       [1., 0., 4., 5.],
       [2., 4., 0., 6.],
       [3., 5., 6., 0.]])

In [84]:
np.logical_and(A[:,0],A[:,1])


array([ True, False,  True,  True, False, False, False])

In [85]:
np.logical_and(np.logical_not(A[:,0]),A[:,1])
np.log10(10000)

4.0

In [86]:
def mutual_info(A, x, u, px, pu):
    n = len(A)
    p_01 = (np.logical_and(np.logical_not(A[:,x]),A[:,u]).sum()+1)/(n+4)
    p_10 = (np.logical_and(A[:,x],np.logical_not(A[:,u])).sum()+1)/(n+4)
    p_11 = (np.logical_and(A[:,x],A[:,u]).sum()+1)/(n+4)
    p_00 = 1 - p_01 - p_10 - p_11
    #print (p_00,p_01,p_10,p_11)
    #We now have all the necessary parameters
    mi = p_00 * np.log10(p_00/(1-px)/(1-pu))+ \
            p_01 * np.log10(p_01/(1-px)/(pu))+ \
            p_10 * np.log10(p_10/(px)/(1-pu))+ \
            p_11 * np.log10(p_11/(px)/(pu))
    return mi

In [87]:
theta_1 = (A.sum(axis = 0)+1)/(m+2)
a = mutual_info(A,1,2,theta_1[1],theta_1[2])

In [88]:
print(a)

0.015772674339962194


In [89]:
G_dense = np.array([[0, 2, 1],[2, 0, 0], [1, 0, 0]])
G_masked = np.ma.masked_values(G_dense, 0)
from scipy.sparse import csr_matrix
G_sparse = csr_matrix(G_dense)

In [90]:
G_sparse

<3x3 sparse matrix of type '<class 'numpy.int32'>'
	with 4 stored elements in Compressed Sparse Row format>

In [91]:
for row_index in range(n-1):
    for column_index in range(row_index+1,n):
        print(row_index,column_index)


0 1
0 2
0 3
1 2
1 3
2 3


In [94]:
MI = np.zeros((n, n))
#Now we build our complete graph with mutual information 
mut_info_list = []
for row_index in range(n-1):
    for column_index in range(row_index+1,n):
        mut_info_list.append(mutual_info(A,row_index,column_index,theta_1[row_index],theta_1[column_index]))

MI[index_of_tri] = mut_info_list
#add that triangle matrix to it own transpose, the you have the symetric matrix
#MI = MI+MI.T
MI

array([[0.        , 0.0226654 , 0.00200688, 0.00200688],
       [0.        , 0.        , 0.01577267, 0.002357  ],
       [0.        , 0.        , 0.        , 0.04984688],
       [0.        , 0.        , 0.        , 0.        ]])

In [95]:
#Maybe is useful to have negative weights
#Now we build our complete graph with mutual information 
mut_info_list = []
for row_index in range(n-1):
    for column_index in range(row_index+1,n):
        mut_info_list.append(-mutual_info(A,row_index,column_index,theta_1[row_index],theta_1[column_index]))

MI[index_of_tri] = mut_info_list
print(MI)

[[ 0.         -0.0226654  -0.00200688 -0.00200688]
 [ 0.          0.         -0.01577267 -0.002357  ]
 [ 0.          0.          0.         -0.04984688]
 [ 0.          0.          0.          0.        ]]


In [97]:
#Then get the min spanig tree
from scipy.sparse.csgraph import minimum_spanning_tree
Tcsr = minimum_spanning_tree(MI)

In [99]:
print(Tcsr)

  (0, 1)	-0.02266540093949237
  (1, 2)	-0.015772674339962194
  (2, 3)	-0.0498468752331349


In [100]:
print(-Tcsr)

  (0, 1)	0.02266540093949237
  (1, 2)	0.015772674339962194
  (2, 3)	0.0498468752331349


In [143]:
from scipy.sparse.csgraph import depth_first_tree
DFS_tree = depth_first_tree(-Tcsr, 2, directed=False)

In [144]:
print(DFS_tree)

  (1, 0)	0.02266540093949237
  (2, 1)	0.015772674339962194
  (2, 3)	0.0498468752331349


In [145]:
DFS_tree.indices

array([0, 1, 3])

In [146]:
DFS_tree.data

array([0.0226654 , 0.01577267, 0.04984688])

In [147]:
DFS_tree.indptr

array([0, 0, 1, 3, 3])

In [148]:
csr_matrix.toarray(DFS_tree)

array([[0.        , 0.        , 0.        , 0.        ],
       [0.0226654 , 0.        , 0.        , 0.        ],
       [0.        , 0.01577267, 0.        , 0.04984688],
       [0.        , 0.        , 0.        , 0.        ]])

## Now Focus on building the CPTs


In [159]:
a =DFS_tree.todok().items()

In [160]:
a

dict_items([((1, 0), 0.02266540093949237), ((2, 1), 0.015772674339962194), ((2, 3), 0.0498468752331349)])

In [164]:
print(DFS_tree)

  (1, 0)	0.02266540093949237
  (2, 1)	0.015772674339962194
  (2, 3)	0.0498468752331349


In [166]:
#Las dependecias que quiero son las siguientes
for arrow in a:
    print (arrow[0][0],arrow[0][1])

1 0
2 1
2 3


In [172]:
#Se podria armar un diccionario con los valores de con lista que contengas p_00 P_01....
#El parametro que se debe hallar es given the tuple ofvariables (a,b) in the DFStree
#We need P(b|a) stores in for the cobinations ab 00 01 10 y 11 which add to 1
#Notice that is convenient to add firs the 11
BN = {}
p_01 = (np.logical_and(np.logical_not(A[:,arrow[0][0]]),A[:,arrow[0][1]]).sum()+1)/(n+4)
p_10 = (np.logical_and(A[:,arrow[0][0]], np.logical_not(A[:,arrow[0][1]])).sum()+1)/(n+4)
p_11 = (np.logical_and(A[:,arrow[0][0]], A[:,arrow[0][1]]).sum()+1)/(n+4)
p_00 = 1 - p_01 - p_10 - p_11
BN[arrow[0]] = np.array([p_00,p_11,p_10, p_01])


In [173]:
print(BN)

{(2, 3): array([0.125, 0.5  , 0.25 , 0.125])}
